In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error

In [3]:
train_data = pd.read_csv('/kaggle/input/30-days-of-ml/train.csv')
test_data = pd.read_csv('/kaggle/input/30-days-of-ml/test.csv')

In [4]:
train_full = train_data.copy()
test_full = test_data.copy()

In [5]:
y = train_full['target']
X = train_full.drop('target', axis = 1)

In [6]:
X.drop('cat9', axis = 1, inplace = True)
test_full.drop('cat9', axis = 1, inplace = True)

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, random_state = 0)

In [8]:
categories = [col for col in X_train.columns if X_train[col].dtype == 'object']

In [9]:
encoder = OrdinalEncoder()

enc_X_train = X_train.copy()
enc_X_valid = X_valid.copy()

enc_X_train[categories] = encoder.fit_transform(X_train[categories])
enc_X_valid[categories] = encoder.transform(X_valid[categories])
test_full[categories] = encoder.transform(test_full[categories])

In [10]:
model = XGBRegressor(n_estimators = 900, learning_rate = 0.03, n_jobs = 2)

model.fit(enc_X_train, y_train)
predictions = model.predict(enc_X_valid)
score = mean_squared_error(y_valid, predictions)
score

0.5225519622430873

In [11]:
final_preds = model.predict(test_full)

In [12]:
output = pd.DataFrame({
    'Id': test_full['id'], 
    'Target': final_preds}
)

In [13]:
output.to_csv('my_submission.csv', index = False)